In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
import cv_bridge

In [2]:
def color(mat,image):
    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 100 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 50 and (sumred/len(mat)) < 100 and (sumgrn/len(mat)) > 80 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'
    
def rectangle(mat,color,image):
    rect = cv.minAreaRect(mat)
    height = rect[1][1]
    width = rect[1][0]
    x = rect[0][0] - width/2
    x = np.int0(x)
    y = rect[0][1] - height/1.7
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    if(color == 'none'):
        cv.putText(image,'noise',(x,y), font, 0.5,(255,255,255),2,cv.LINE_AA)
    elif((1.5*width < height and 3.5*width > height) or (1.5*height < width and 3.5*height > width)):
        cv.putText(image,color+' buoy',(x,y), font, 0.5,(255,255,255),2,cv.LINE_AA)
    box = cv.boxPoints(rect)
    box = np.int0(box)
    cv.drawContours(image,[box],0,(255,0,255),2)
    return image

def pp(image):
    image = np.array(image[0:800][0:515])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    return(image)

def transform(image):
    
    img = pp(image)
    img2 = dbscan(img)
    
    return img2

def dbscan(image): 
    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret,thresh1 = cv.threshold(img,40,255,cv.THRESH_BINARY)

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
    
    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=15).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)


        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])
        
        x = 0

        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

            i = 0
            x = 0
            l = []

            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    l = np.array(l)
                    clr = color(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image)
                    l = []
            image = tb   
    return image

In [3]:
def read(x):    
    img = cv.imread('images/left' + str(x).zfill(4) + '.jpg',1)
    result1 = transform(img)
    cv.imwrite('output_images/hsv/hsv' + str(x).zfill(4) + '.jpg', result1)

In [4]:
def callback(image):
    starttime = time.time()
    bridge = cv_bridge.CvBridge()
    frame = image.header.seq
    image = bridge.imgmsg_to_cv2(image, desired_encoding="passthrough")
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    image = transform(image)
    cv.imshow("left/image_raw", image)
    cv.waitKey(1)
    print("That took %d seconds", format(time.time()-starttime))

In [ ]:
#new main
if __name__ == '__main__':
    rospy.init_node('liveproc')
    rospy.Subscriber("stereo/left/image_raw", Image, callback)
    rospy.spin()

#old main
"""if __name__=='__main__':
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    starttime = time.time()
    
    pool.map(read,range(2728),30)
    pool.close()
    pool.join()
    
    print('That took {} seconds',format(time.time() - starttime))"""

('That took %d seconds', '10.9765880108')
('That took %d seconds', '0.0769860744476')
('That took %d seconds', '0.0782098770142')
('That took %d seconds', '0.0751619338989')
('That took %d seconds', '0.0724129676819')
('That took %d seconds', '0.0860819816589')
('That took %d seconds', '0.0812528133392')
('That took %d seconds', '0.075560092926')
('That took %d seconds', '0.0827689170837')
('That took %d seconds', '0.0851621627808')
('That took %d seconds', '0.08238697052')
('That took %d seconds', '0.0721299648285')
('That took %d seconds', '0.07847905159')
('That took %d seconds', '0.0719759464264')
('That took %d seconds', '0.0857388973236')
('That took %d seconds', '0.0772938728333')
('That took %d seconds', '0.0798029899597')
('That took %d seconds', '0.0688560009003')
('That took %d seconds', '0.0784230232239')
('That took %d seconds', '0.0960409641266')
('That took %d seconds', '0.0695369243622')
('That took %d seconds', '0.0949637889862')
('That took %d seconds', '0.07577705383

('That took %d seconds', '0.0912299156189')
('That took %d seconds', '0.0941500663757')
('That took %d seconds', '0.0725400447845')
('That took %d seconds', '0.075001001358')
('That took %d seconds', '0.07634806633')
('That took %d seconds', '0.078047990799')
('That took %d seconds', '0.0855679512024')
('That took %d seconds', '0.0697309970856')
('That took %d seconds', '0.0695469379425')
('That took %d seconds', '0.0669860839844')
('That took %d seconds', '0.0684649944305')
('That took %d seconds', '0.0668289661407')
('That took %d seconds', '0.070063829422')
('That took %d seconds', '0.0657479763031')
('That took %d seconds', '0.0727858543396')
('That took %d seconds', '0.0652270317078')
('That took %d seconds', '0.0653159618378')
('That took %d seconds', '0.0614328384399')
('That took %d seconds', '0.0621328353882')
('That took %d seconds', '0.0652649402618')
('That took %d seconds', '0.0634450912476')
('That took %d seconds', '0.0643289089203')
('That took %d seconds', '0.060502052

('That took %d seconds', '0.0767631530762')
('That took %d seconds', '0.0764830112457')
('That took %d seconds', '0.0704829692841')
('That took %d seconds', '0.0800089836121')
('That took %d seconds', '0.0826358795166')
('That took %d seconds', '0.0676980018616')
('That took %d seconds', '0.0738739967346')
('That took %d seconds', '0.0709459781647')
('That took %d seconds', '0.0689311027527')
('That took %d seconds', '0.0689768791199')
('That took %d seconds', '0.0701820850372')
('That took %d seconds', '0.0680811405182')
('That took %d seconds', '0.0781099796295')
('That took %d seconds', '0.0786809921265')
('That took %d seconds', '0.0611779689789')
('That took %d seconds', '0.064150094986')
('That took %d seconds', '0.0652959346771')
('That took %d seconds', '0.0662059783936')
('That took %d seconds', '0.0731148719788')
('That took %d seconds', '0.0669059753418')
('That took %d seconds', '0.0719559192657')
('That took %d seconds', '0.0803110599518')
('That took %d seconds', '0.06297

('That took %d seconds', '0.0704410076141')
('That took %d seconds', '0.0708711147308')
('That took %d seconds', '0.0621590614319')
('That took %d seconds', '0.0762867927551')
('That took %d seconds', '0.0747380256653')
('That took %d seconds', '0.0757730007172')
('That took %d seconds', '0.076033115387')
('That took %d seconds', '0.0827589035034')
('That took %d seconds', '0.0839829444885')
('That took %d seconds', '0.0900502204895')
('That took %d seconds', '0.0737490653992')
('That took %d seconds', '0.0713639259338')
('That took %d seconds', '0.0843069553375')
('That took %d seconds', '0.073410987854')
('That took %d seconds', '0.0821540355682')
('That took %d seconds', '0.0896949768066')
('That took %d seconds', '0.0706000328064')
('That took %d seconds', '0.0745787620544')
('That took %d seconds', '0.0860340595245')
('That took %d seconds', '0.082827091217')
('That took %d seconds', '0.0799510478973')
('That took %d seconds', '0.077455997467')
('That took %d seconds', '0.07573485

('That took %d seconds', '0.0996999740601')
('That took %d seconds', '0.0953068733215')
('That took %d seconds', '0.0855391025543')
('That took %d seconds', '0.0823979377747')
('That took %d seconds', '0.0886499881744')
('That took %d seconds', '0.0806350708008')
('That took %d seconds', '0.0798020362854')
('That took %d seconds', '0.0863289833069')
('That took %d seconds', '0.082396030426')
('That took %d seconds', '0.0818660259247')
('That took %d seconds', '0.0811009407043')
('That took %d seconds', '0.0782420635223')
('That took %d seconds', '0.0817110538483')
('That took %d seconds', '0.0792610645294')
('That took %d seconds', '0.0874300003052')
('That took %d seconds', '0.0786800384521')
('That took %d seconds', '0.0951759815216')
('That took %d seconds', '0.0867900848389')
('That took %d seconds', '0.0920579433441')
('That took %d seconds', '0.0974590778351')
('That took %d seconds', '0.114382982254')
('That took %d seconds', '0.0918328762054')
('That took %d seconds', '0.093768

...

('That took %d seconds', '0.0185010433197')
('That took %d seconds', '0.0199389457703')
('That took %d seconds', '0.0197529792786')
('That took %d seconds', '0.0202090740204')
('That took %d seconds', '0.0197179317474')
('That took %d seconds', '0.0193650722504')
('That took %d seconds', '0.0182971954346')
('That took %d seconds', '0.0190711021423')
('That took %d seconds', '0.018935918808')
('That took %d seconds', '0.0198540687561')
('That took %d seconds', '0.0188238620758')
('That took %d seconds', '0.0196948051453')
('That took %d seconds', '0.0191180706024')
('That took %d seconds', '0.02024102211')
('That took %d seconds', '0.0225319862366')
('That took %d seconds', '0.0193300247192')
('That took %d seconds', '0.0193839073181')
('That took %d seconds', '0.0201458930969')
('That took %d seconds', '0.0191130638123')
('That took %d seconds', '0.019474029541')
('That took %d seconds', '0.019679069519')
('That took %d seconds', '0.0188648700714')
('That took %d seconds', '0.019042015

('That took %d seconds', '0.0200741291046')
('That took %d seconds', '0.018501996994')
('That took %d seconds', '0.0202651023865')
('That took %d seconds', '0.0212230682373')
('That took %d seconds', '0.0185699462891')
('That took %d seconds', '0.0178079605103')
('That took %d seconds', '0.019021987915')
('That took %d seconds', '0.0187320709229')
('That took %d seconds', '0.0198049545288')
('That took %d seconds', '0.022369146347')
('That took %d seconds', '0.0204129219055')
('That took %d seconds', '0.0178070068359')
('That took %d seconds', '0.0186910629272')
('That took %d seconds', '0.017657995224')
('That took %d seconds', '0.0184118747711')
('That took %d seconds', '0.0192289352417')
('That took %d seconds', '0.0192258358002')
('That took %d seconds', '0.0188689231873')
('That took %d seconds', '0.0189568996429')
('That took %d seconds', '0.0197479724884')
('That took %d seconds', '0.0190181732178')
('That took %d seconds', '0.0194549560547')
('That took %d seconds', '0.01821494

('That took %d seconds', '0.0225460529327')
('That took %d seconds', '0.0236530303955')
('That took %d seconds', '0.0214281082153')
('That took %d seconds', '0.0205810070038')
('That took %d seconds', '0.0219309329987')
('That took %d seconds', '0.0206389427185')
('That took %d seconds', '0.0211038589478')
('That took %d seconds', '0.0218229293823')
('That took %d seconds', '0.0207061767578')
('That took %d seconds', '0.0218470096588')
('That took %d seconds', '0.024286031723')
('That took %d seconds', '0.0227401256561')
('That took %d seconds', '0.0214941501617')
('That took %d seconds', '0.0208821296692')
('That took %d seconds', '0.0219569206238')
('That took %d seconds', '0.0219779014587')
('That took %d seconds', '0.0206689834595')
('That took %d seconds', '0.0204839706421')
('That took %d seconds', '0.0203530788422')
('That took %d seconds', '0.0231139659882')
('That took %d seconds', '0.0202770233154')
('That took %d seconds', '0.020693063736')
('That took %d seconds', '0.021011

('That took %d seconds', '0.0179150104523')
('That took %d seconds', '0.017128944397')
('That took %d seconds', '0.0184881687164')
('That took %d seconds', '0.0172469615936')
('That took %d seconds', '0.017541885376')
('That took %d seconds', '0.0178239345551')
('That took %d seconds', '0.0180079936981')
('That took %d seconds', '0.0184280872345')
('That took %d seconds', '0.0197958946228')
('That took %d seconds', '0.0174539089203')
('That took %d seconds', '0.0181331634521')
('That took %d seconds', '0.0188431739807')
('That took %d seconds', '0.0181219577789')
('That took %d seconds', '0.0203759670258')
('That took %d seconds', '0.0176198482513')
('That took %d seconds', '0.0184059143066')
('That took %d seconds', '0.0184240341187')
('That took %d seconds', '0.0195951461792')
('That took %d seconds', '0.0200588703156')
('That took %d seconds', '0.0191857814789')
('That took %d seconds', '0.0208899974823')
('That took %d seconds', '0.0199961662292')
('That took %d seconds', '0.019418

('That took %d seconds', '0.0252850055695')
('That took %d seconds', '0.0204701423645')
('That took %d seconds', '0.0202271938324')
('That took %d seconds', '0.0208129882812')
('That took %d seconds', '0.0183489322662')
('That took %d seconds', '0.0208621025085')
('That took %d seconds', '0.026242017746')
('That took %d seconds', '0.0251460075378')
('That took %d seconds', '0.0207090377808')
('That took %d seconds', '0.0209767818451')
('That took %d seconds', '0.0202879905701')
('That took %d seconds', '0.0197138786316')
('That took %d seconds', '0.0202810764313')
('That took %d seconds', '0.0218648910522')
('That took %d seconds', '0.0195748806')
('That took %d seconds', '0.0214989185333')
('That took %d seconds', '0.0206439495087')
('That took %d seconds', '0.0199618339539')
('That took %d seconds', '0.0225048065186')
('That took %d seconds', '0.0210158824921')
('That took %d seconds', '0.0220909118652')
('That took %d seconds', '0.02161693573')
('That took %d seconds', '0.0220000743